# 신용카드 사용자 연체 예측 AI 경진대회

### 데이터 설명
- `train.csv`: 신용카드 사용자들의 개인 신상정보 (26457, 20)   
- `test.csv`: credit 열 미포함 (10000, 19)   
- `sample_submission.csv`: 정답 제출 파일 (10000, 4)

### 데이터 변수 설명
- `index`
- `gender`: 성별
- `car`: 차량 소유 여부
- `reality`: 부동산 소유 여부
- `child_num`: 자녀 수
- `income_total`: 연간 소득
- `income_type`: 소득 분류 ['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']
- `edu_type`: 교육 수준 ['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']
- `family_type`: 결혼 여부 ['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']
- `house_type`: 생활 방식 ['Municipal apartment', 'House / apartment', 'With parents', 'Co-op apartment', 'Rented apartment', 'Office apartment']
- `DAYS_BIRTH`: 출생일 (데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미)
- `DAYS_EMPLOYED`: 업무 시작일 (데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미, 양수 값은 고용되지 않은 상태를 의미함)
- `FLAG_MOBIL`: 핸드폰 소유 여부
- `work_phone`: 업무용 전화 소유 여부
- `phone`: 전화 소유 여부
- `email`: 이메일 소유 여부
- `occyp_type`: 직업 유형													
- `family_size`: 가족 규모
- `begin_month`: 신용카드 발급 월 (데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미)
- `credit`: 사용자의 신용카드 대금 연체를 기준으로 한 신용도 (=> 낮을 수록 높은 신용의 신용카드 사용자를 의미함)

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/train.csv')
df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


## 데이터 전처리

In [2]:
df = pd.read_csv('data/train.csv')
df[~((df['occyp_type'].isnull()) & (df['DAYS_EMPLOYED'] < 0))]
df['occyp_type'] = df['occyp_type'].fillna(0)
df['gender'].replace({'M':0,'F':1}, inplace=True)
df['car'].replace({'N':0,'Y':1}, inplace=True)
df['reality'].replace({'N':0,'Y':1}, inplace=True)

income_dict = {'Working': 0, 'Commercial associate': 1, 'Pensioner': 2, 'State servant': 3, 'Student': 4}
edu_dict = {'Secondary / secondary special': 0, 'Higher education': 1, 'Incomplete higher': 2, 'Lower secondary': 3, 'Academic degree': 4}
family_dict = {'Married': 0, 'Single / not married': 1, 'Civil marriage': 2, 'Separated': 3, 'Widow': 4}
house_dict = {'House / apartment': 0, 'With parents': 1, 'Municipal apartment': 2, 'Rented apartment': 3, 'Office apartment': 4, 'Co-op apartment': 5}
occyp_dict = {'Laborers': 1, 'Core staff': 2, 'Sales staff': 3, 'Managers': 4, 'Drivers': 5, 'High skill tech staff': 6, 'Accountants': 7,
              'Medicine staff': 8, 'Cooking staff': 9, 'Security staff': 10, 'Cleaning staff': 11, 'Private service staff': 12, 'Low-skill Laborers': 13,
              'Waiters/barmen staff': 14, 'Secretaries': 15, 'Realty agents': 16, 'HR staff': 17, 'IT staff': 18}

df['income_type'].replace(income_dict, inplace=True)
df['edu_type'].replace(edu_dict, inplace=True)
df['family_type'].replace(family_dict, inplace=True)
df['house_type'].replace(house_dict, inplace=True)
df['occyp_type'].replace(occyp_dict, inplace=True)

In [3]:
df.head()

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,0,1,0,0,0,202500.0,1,1,0,2,-13899,-4709,1,0,0,0,0,2.0,-6.0,1.0
1,1,1,0,1,1,247500.0,1,0,2,0,-11380,-1540,1,0,0,1,1,3.0,-5.0,1.0
2,2,0,1,1,0,450000.0,0,1,0,0,-19087,-4434,1,0,1,0,4,2.0,-22.0,2.0
3,3,1,0,1,0,202500.0,1,0,0,0,-15088,-2092,1,0,1,0,3,2.0,-37.0,0.0
4,4,1,1,1,0,157500.0,3,1,0,0,-15037,-2105,1,0,0,0,4,2.0,-26.0,2.0


In [4]:
df.set_index('index').to_csv('data_prep.csv')